# Algoritmo para aumentar o SESA para o projeto final de ML

In [1]:
import librosa
import efeitos
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
from joblib import Parallel, delayed

#DEFININDO ALGUNS PARÂMETROS PARA PLOTAR GRÁFICOS
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
params = {'figure.figsize': [10, 5], 
          'axes.labelsize': 13,
          'axes.titlesize':17, 
          'font.size': 13,
          'legend.fontsize': 13, 
          'xtick.labelsize': 11, 
          'ytick.labelsize': 11
         }
plt.rcParams.update(params)

## Aumentando o dataset de treino

Serão utilizados seis datasets de treinamento diferentes:

1) Apenas com os áudios originais

2) Com os originais e com os deslocados no tempo

3) Com os originais e com os alterados no pitch

4) Com os originais e com os alterados na velocidade

5) Com os originais e com ruído branco

6) Todos juntos

In [ ]:
def aumentarDatasetTreino(nomeAudioTreino, diretorioOrigem, diretorioDestino):
    
    # ABRINDO O AUDIO
    sinalOriginal, freqAmostragem = librosa.core.load(diretorioOrigem + nomeAudioTreino, sr=None, mono=True)
    
    # NORMALIZANDO PELA ENERGIA
    sinalOriginal = efeitos.normalizarEnergia(sinalOriginal)
    
    # APLICANDO OS EFEITOS
    sinalVelocidade, velocidade = efeitos.alterarVelocidadeAleatoriamente(sinalOriginal)
    sinalRuido, snr             = efeitos.adicionarRuidoBrancoAleatoriamente(sinalOriginal)
    sinalPitch, pitch           = efeitos.mudarPitchAleatoriamente(sinalOriginal, freqAmostragem)
    sinalTempo, tempo           = efeitos.deslocarTempoAleatoriamente(sinalOriginal)
    
    # SALVANDO NO DESTINO (TEM QUE SALVAR O ORIGINAL TB)
    librosa.output.write_wav(diretorioDestino + nomeAudioTreino[:-4] + "_original.wav", sinalOriginal, freqAmostragem)
    librosa.output.write_wav(diretorioDestino + nomeAudioTreino[:-4] + "_velocidade.wav", sinalVelocidade, freqAmostragem)
    librosa.output.write_wav(diretorioDestino + nomeAudioTreino[:-4] + "_ruido.wav", sinalRuido, freqAmostragem)
    librosa.output.write_wav(diretorioDestino + nomeAudioTreino[:-4] + "_pitch.wav", sinalPitch, freqAmostragem)
    librosa.output.write_wav(diretorioDestino + nomeAudioTreino[:-4] + "_tempo.wav", sinalTempo, freqAmostragem)
    
    # RETORNO OS PARAMETROS DE CADA EFEITO
    return velocidade, snr, pitch, tempo

In [ ]:
diretorioOrigem  = "/home/dimi/Downloads/Datasets/SESA/SESA_Original/train/"
diretorioDestino = "/home/dimi/Downloads/Datasets/SESA/SESA_Aumentado_Projeto_Final_ML/train/"

arquivosOrigem = np.array(os.listdir(diretorioOrigem))
#arquivosOrigem = arquivosOrigem[:10]

resposta = Parallel(n_jobs=-1, verbose=100)(delayed(aumentarDatasetTreino)(arquivoAtual, diretorioOrigem, diretorioDestino) for arquivoAtual in arquivosOrigem)
arrayVelocidades, arraySNRs, arrayPitchs, arrayTempos = zip(*resposta)

In [ ]:
bins = 50

plt.hist(arrayPitchs, bins, histtype="stepfilled")
plt.title("Distribuição das alterações de pitch no dataest de treinamento")
plt.grid(alpha=0.3)
plt.xlabel("Pitch Shift")
plt.savefig("distribuicao_treinamento_pitch.pdf", format='pdf', bbox_inches = 'tight')
plt.show()

plt.hist(arraySNRs, bins, histtype="stepfilled")
plt.title("Distribuição das SNRs no dataest de treinamento")
plt.grid(alpha=0.3)
plt.xlabel("SNR (dB)")
plt.savefig("distribuicao_treinamento_ruido.pdf", format='pdf', bbox_inches = 'tight')
plt.show()

plt.hist(arrayVelocidades, bins, histtype="stepfilled")
plt.title("Distribuição das velocidades de reprodução no dataset de treinamento")
plt.grid(alpha=0.3)
plt.xlabel("Velocidade de Reprodução")
plt.savefig("distribuicao_treinamento_velocidade.pdf", format='pdf', bbox_inches = 'tight')
plt.show()

plt.hist(arrayTempos, bins, histtype="stepfilled")
plt.title("Distribuição dos percentuais de deslocamentos temporais no dataset de treinamento")
plt.grid(alpha=0.3)
plt.xlabel("Percentual de Deslocamento Temporal")
plt.savefig("distribuicao_treinamento_tempo.pdf", format='pdf', bbox_inches = 'tight')
plt.show()

### Aumentando o dataset de teste

Cada áudio na origem vai gerar oito áudios no destino:

1) Original

2) Com deslocamento temporal

3) Com mudança de pitch

4) Com adição de ruído branco

5) Com alteração na velocidade de reprodução

6) Com dois efeitos misturados (selecionados aleatoriamente)

7) Com três efeitos misturados (selecionados aleatoriamente)

8) Com todos os efeitos

In [40]:
def aumentarDatasetTeste(nomeAudioTeste, diretorioOrigem, diretorioDestino):
    
    # ABRINDO O AUDIO
    sinalOriginal, freqAmostragem = librosa.core.load(diretorioOrigem + nomeAudioTeste, sr=None, mono=True)
    
    # NORMALIZANDO PELA ENERGIA
    sinalOriginal = efeitos.normalizarEnergia(sinalOriginal)
    
    # APLICANDO OS EFEITOS SINGULARES
    sinalVelocidade, __ = efeitos.alterarVelocidadeAleatoriamente(sinalOriginal)
    sinalRuido, __      = efeitos.adicionarRuidoBrancoAleatoriamente(sinalOriginal)
    sinalPitch, __      = efeitos.mudarPitchAleatoriamente(sinalOriginal, freqAmostragem)
    sinalTempo, __      = efeitos.deslocarTempoAleatoriamente(sinalOriginal)
    librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_original.wav", sinalOriginal, freqAmostragem)
    librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_velocidade.wav", sinalVelocidade, freqAmostragem)
    librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_ruido.wav", sinalRuido, freqAmostragem)
    librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_pitch.wav", sinalPitch, freqAmostragem)
    librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_tempo.wav", sinalTempo, freqAmostragem)
    
    # APLICANDO DOIS EFEITOS
    escolha = np.random.randint(0, 6)    
    if escolha == 0: # efeitos == [1,2]
        print("Entrou em 0")
        sinalDois, __ = efeitos.alterarVelocidadeAleatoriamente(sinalOriginal)
        sinalDois, __ = efeitos.adicionarRuidoBrancoAleatoriamente(sinalDois)
        librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_velocidade_ruido.wav", sinalDois, freqAmostragem)
    elif escolha == 1: # efeitos == [1,3]
        print("Entrou em 1")
        sinalDois, __ = efeitos.alterarVelocidadeAleatoriamente(sinalOriginal)
        sinalDois, __ = efeitos.mudarPitchAleatoriamente(sinalDois, freqAmostragem)
        librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_velocidade_pitch.wav", sinalDois, freqAmostragem)
    elif escolha == 2: # efeitos == [1,4]
        print("Entrou em 2")
        sinalDois, __ = efeitos.alterarVelocidadeAleatoriamente(sinalOriginal)
        sinalDois, __ = efeitos.deslocarTempoAleatoriamente(sinalDois)
        librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_velocidade_tempo.wav", sinalDois, freqAmostragem)
    elif escolha == 3: # efeitos == [2,3]
        print("Entrou em 3")
        sinalDois, __ = efeitos.adicionarRuidoBrancoAleatoriamente(sinalOriginal)
        sinalDois, __ = efeitos.mudarPitchAleatoriamente(sinalDois, freqAmostragem)
        librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_ruido_pitch.wav", sinalDois, freqAmostragem)
    elif escolha == 4: # efeitos == [2,4]
        print("Entrou em 4")
        sinalDois, __ = efeitos.adicionarRuidoBrancoAleatoriamente(sinalOriginal)
        sinalDois, __ = efeitos.deslocarTempoAleatoriamente(sinalDois)
        librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_ruido_tempo.wav", sinalDois, freqAmostragem)
    else: # efeitos == [3,4]
        print("Entrou no else")
        sinalDois, __ = efeitos.mudarPitchAleatoriamente(sinalOriginal, freqAmostragem)
        sinalDois, __ = efeitos.deslocarTempoAleatoriamente(sinalDois)
        librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_pitch_tempo.wav", sinalDois, freqAmostragem)
    
    # APLICANDO TRES EFEITOS
    escolha = np.random.randint(0, 4)
    if escolha == 0: # efeitos == [1,2,3]
        sinalTres, __ = efeitos.alterarVelocidadeAleatoriamente(sinalOriginal)
        sinalTres, __ = efeitos.adicionarRuidoBrancoAleatoriamente(sinalTres)
        sinalTres, __ = efeitos.mudarPitchAleatoriamente(sinalTres, freqAmostragem)
        librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_velocidade_ruido_pitch.wav", sinalTres, freqAmostragem)
    elif escolha == 1: # efeitos == [1,2,4]
        sinalTres, __ = efeitos.alterarVelocidadeAleatoriamente(sinalOriginal)
        sinalTres, __ = efeitos.adicionarRuidoBrancoAleatoriamente(sinalTres)
        sinalTres, __ = efeitos.deslocarTempoAleatoriamente(sinalTres)
        librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_velocidade_ruido_tempo.wav", sinalTres, freqAmostragem)
    elif escolha == 2: # efeitos == [1,3,4]
        sinalTres, __ = efeitos.alterarVelocidadeAleatoriamente(sinalOriginal)
        sinalTres, __ = efeitos.mudarPitchAleatoriamente(sinalTres, freqAmostragem)
        sinalTres, __ = efeitos.deslocarTempoAleatoriamente(sinalTres)
        librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_velocidade_pitch_tempo.wav", sinalTres, freqAmostragem)
    else: # efeitos == [2,3,4]
        sinalTres, __ = efeitos.adicionarRuidoBrancoAleatoriamente(sinalOriginal)
        sinalTres, __ = efeitos.mudarPitchAleatoriamente(sinalTres, freqAmostragem)
        sinalTres, __ = efeitos.deslocarTempoAleatoriamente(sinalTres)
        librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_ruido_pitch_tempo.wav", sinalTres, freqAmostragem)
    
    # APLICANDO TODOS OS EFEITOS
    sinalTodos, __ = efeitos.alterarVelocidadeAleatoriamente(sinalOriginal)
    sinalTodos, __ = efeitos.adicionarRuidoBrancoAleatoriamente(sinalTodos)
    sinalTodos, __ = efeitos.mudarPitchAleatoriamente(sinalTodos, freqAmostragem)
    sinalTodos, __ = efeitos.deslocarTempoAleatoriamente(sinalTodos)
    librosa.output.write_wav(diretorioDestino + nomeAudioTeste[:-4] + "_velocidade_ruido_pitch_tempo.wav", sinalTodos, freqAmostragem)  

In [42]:
diretorioOrigem  = "/home/dimi/Downloads/Datasets/SESA/SESA_Original/test/"
diretorioDestino = "/home/dimi/Downloads/Datasets/SESA/SESA_Aumentado_Projeto_Final_ML/test/"

arquivosOrigem = np.array(os.listdir(diretorioOrigem))
#arquivosOrigem = arquivosOrigem[:10]

Parallel(n_jobs=-1, verbose=100)(delayed(aumentarDatasetTeste)(arquivoAtual, diretorioOrigem, diretorioDestino) for arquivoAtual in arquivosOrigem)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]